### Welcome

This is the first section of the introductory GeoIPS tutorial, which includes running GeoIPS 
using the CLI, and creating your first plugin! 


Link to these slides 
https://github.com/NRLMMD-GEOIPS/presentations

### Tutorial Scope

This tutorial does not address running GeoIPS in near real-time. GeoIPS plugins 
are intended to be developed and tested on a specific dataset, and setting up the 
real-time processing infrastructure is a separate conversation. 

This tutorial focuses on: 

  - Installing GeoIPS 
  - Running GeoIPS 
  - Product development and testing 


### To get the most out of this tutorial… 

- [Aug 2024 - Intro GeoIPS Tutorial - Google Slides](https://docs.google.com/presentation/d/151tR2hycoM3WqC9-7s9laMcCTZ9StY2fVLTpA_FwYBs/edit#slide=id.g2e736e3f9ff_3_94) 
- Please keep these slides up to copy/paste from as needed throughout the 
tutorial 
- Please copy/paste rather than typing everything word for word! 
  - Trick - Triple click to select a full code block for copy/paste! 
- [NRLMMD-GEOIPS/template_basic_plugin workshop-2024-beginner-solutions](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2024-beginner-solutions) 
- If all else fails, you can access the solutions on github.com 

## System requirements

- **CPU:** 1 CPU
- **RAM:** 40GB as the notebook is written, but could use more if modified.
  Reading the entire full-disk ABI image can take up to 100GB.
- **Disk Space:** 3GB storage space.

***To see the default storage location for your system, run the cell below.***
To change the default location, change the value of `tmp_root` in the following cell.

In [ ]:
%%bash

# Run this if this notebook has been ran on your system previously -- doesn't hurt if you haven't
pip uninstall -y cool_plugins


## Setting up your Environment

**If you don't have an environment already set up, please follow the steps 2 and 3 of <br />
the instructions linked [here](https://nrlmmd-geoips.github.io/geoips/getting-started/installing/index.html) based on the architecture of your machine.**

You can quit following the instructions after you've activated your geoips environment via <br />
``conda activate geoips``.

If running locally, run the python module below via `python setup_environment.py` to set <br />
your environment. This must be done manually in the terminal.

```bash
python setup_environment.py
```

**After** running the python command above in the terminal, run the code block below.


In [ ]:
import os

import dotenv

dotenv.load_dotenv("./.env", override=True)

if not os.path.exists(os.environ["GEOIPS_TESTDATA_DIR"]):
    os.makedirs(os.environ["GEOIPS_TESTDATA_DIR"])
if not os.path.exists(os.environ["GEOIPS_OUTDIRS"]):
    os.makedirs(os.environ["GEOIPS_OUTDIRS"])

### Installing Appropriate Test Datasets

For this tutorial, you'll need GeoIPS test datasets that can be used to produce imagery 
or scientific datasets. 

If just attending the beginner tutorial, you'll need:

  - `test_data_clavrx`

These datasets might take a while to download so go ahead and download the appropriate 
datasets based on the following commands,.

In [ ]:
%%bash

# This is needed for both the beginner and advanced tutorial
geoips config install test_data_clavrx --outdir $GEOIPS_TESTDATA_DIR

### Introduction to GeoIPS

GeoIPS is a plugin-based system for processing geolocated data: 
  - produce imagery in several formats (most often PNG). 
  - produce output data products in NetCDF4 format. 
  - extended to add other output formats via plugins. 

![1111𝝁 infrared imagery](./images/conus_infrared.png)

![Himawari-9 CLAVR-x Cloud-Top-Height](./images/ahi_cloud_top_height.png)

![ABI CLAVR-x Cloud Top Height](./images/abi_cloud_top_height.png)

GeoIPS is almost entirely composed of plugins:

  - GeoIPS can be extended by developing new plugins in external python packages. 
  - No need to edit the main GeoIPS code to add new functionality. 
  - Most types of functionality in GeoIPS can be extended (and if something can’t be 
  extended, and you think it should, let us know!). 


### Vocabulary

**YAML**
  - "Yet Another Markdown Language" 
  - A human-readable data serialization language that is often used for writing configuration files 

**Plugin**
  - A Python module or YAML file that defines GeoIPS functionality 
  - Stored in an installable Python package that registers its plugin payload with GeoIPS 

**Interface**
  - A class of Python plugins that modify the same type of functionality within GeoIPS 
  (e.g., “the algorithms interface” or “the colormappers interface”) 
  
**Family**
  - A subset of an interface whose plugins accept different sets of arguments/properties 
  - Planning to deprecate this 'artifact' in the near future.


### Commonly used GeoIPS Plugin Types 

**algorithm**
  - Implements a function that modifies data and outputs new data

**colormapper**
  - Defines a method of applying a colormap to imagery

**feature_annotator**
  - Defines how to plot map features (e.g. coastlines, borders, rivers, etc.)

**gridline_annotator**
  - Defines how to plot gridlines and labels

**interpolator**
  - Defines a method of interpolating data to a sector

**output_formatter**
  - Defines a method for plotting imagery or outputting a data file 

**procflow**
  - Defines the order of operations to use when producing a product from GeoIPS 

**product**
  - Defines how to produce a specific product as a combination of other plugins
  - Uses other plugins (e.g. algorithm, colormapper, interpolator)

**product_defaults**
  - Defines a default set of plugins and arguments for producing a specific type of product 
  - Can be reused across multiple products to allow consistency between similar products 

**reader**
  - Defines a data reader 

**sector**
  - Describes an domain for reprojection of data 

## Hands On: Modify a plugin template to create your own installable plugin package  


### Get the Template Repo

Run the following series of commands to get the template repository.

In [ ]:
%%bash

cd $GEOIPS_PACKAGES_DIR
git clone --no-tags --single-branch $GEOIPS_REPO_URL/template_basic_plugin.git
# Rename your package
mv template_basic_plugin/ $MY_PKG_NAME
cd $MY_PKG_NAME
# No longer point to github.com template_basic_plugin.git
git remote remove origin

### Update the Package Name

Ensure you’re in your package directory and look around.

This repository is set up with a working installable plugin called "my_package", so we 
just need to swap out the name and build upon what's already there! 

![Package listing](./images/cool_plugins_top_level.png)

This repository is set up with a working installable plugin called "my_package", so we 
just need to swap out the name and build upon what's already there!

### Structure of a Plugin Package

Rename the default plugin package directory to your new package name

In [ ]:
%%bash

cd $MY_PKG_DIR
git mv my_package $MY_PKG_NAME
tree -L 2

### Your directory structure should now look like this:

![cool_plugins directory structure](./images/cool_plugins_directory_structure.png)

In [ ]:
%%bash

cd $MY_PKG_DIR/cool_plugins
tree

![plugins directory structure](./images/plugins_directory_structure.png)

### Update Pertinent Files

1. Update README.md (`vim README.md`)
  - Find/replace all occurrences of @package@  with your package name 
  - Vim Tip :%s/@package@/cool_plugins/g 
  
**Note**: The @ symbols are for ease of searching, take them out when you put your 
package name in!

2. Update pyproject.toml (`vim pyproject.toml`, more on this in soon.)
  - Find/replace all occurrences of my_package  with your package name 

3. Add and commit your changes.

#### Note

Due to the nature of Jupyter Notebooks, we are not able to edit files in place (that is <br />
opening them up and manually editing files (such as YAML or TOML)). Instead we'll <br />
update these files with Python. Keep in mind if you were running this notebook locally <br />
via an IDE, you'd be able to edit these files manually rather than using Python.

In [ ]:
"""Overwrite cool_plugins' pyproject.toml and README.md with correct contents."""

import os

with open("./updated_files/pyproject.toml", "r") as rf:
    toml_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/pyproject.toml", "w") as wf:
    wf.writelines(toml_lines)

with open("./updated_files/README.md", "r") as rf:
    md_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/README.md", "w") as wf:
    wf.writelines(md_lines)

In [ ]:
%%bash

cd $MY_PKG_DIR
git add README.md pyproject.toml
git commit -m "Updated name of template plugin package to mine"

4. Install your package (-e means `editable` so we can edit the package after it is 
installed and changes will be reflected in the installed package)

Maybe mention a symlink -- similar to if you've symlinked your package to python installation path (site-packages)

In [ ]:
%%bash

pip install -e $MY_PKG_DIR

5. Register your plugins so GeoIPS knows where to find them

In [ ]:
%%bash

geoips config create-registries

### See what you just installed 

List all installed packages:

In [ ]:
%%bash

geoips list packages

List all installed plugins:

In [ ]:
%%bash

geoips list plugins

### A bit about pyproject.toml

Installing Python packages requires metadata that describes the package and how to 
install it. 

`pyproject.toml` defines this information for pip, including: 
  - Package name, version, description, license, etc. 
  - Which files should be contained in the package when installed 
  - How to build the package 

We make GeoIPS aware of our package using the `geoips.plugin_packages` namespace.
This allows GeoIPS to find all plugins within packages registered to this namespace.

GeoIPS automatically identifies all plugins defined within a plugin package via a plugin
registry. You can manually create these files via `geoips config create-registries`, 
however, GeoIPS will automatically create these files if a requested plugin cannot be
found. This usually occurs the first time GeoIPS is initialized.

**NOTE** for plugin registries to write successfully: 

1. All installed  plugin names within a given interface must be unique 

2. All installed plugins must be formatted and defined correctly 

We will make use of this more later! Modify plugin template solutions on 
[NRLMMD github.com](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2023-solutions).

```
[tool.poetry.plugins."geoips.plugin_packages"]
"cool_plugins" = "cool_plugins" 
``` 

## GeoIPS Command Line Interface (CLI) Tutorial

Please follow this [Jupyter Notebook](./CLI_Tutorial.ipynb) for instructions on how to 
make use of the GeoIPS CLI.

## Hands on: Create a Product Plugin (products  YAML-based interface) 

Solutions on 
[NRLMMD github.com](https://github.com/NRLMMD-GEOIPS/template_basic_plugin/tree/workshop-2023-solutions).

### First Product Plugin: Cloud Top Height (CTH) from CLAVR-x data

- Copy the existing product plugin to a new file to modify 

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/products
cp amsr2_using_product_defaults.yaml my_clavrx_products.yaml

- Edit my_clavrx_products.yaml properties (I.e. `vim my_clavrx_products.yaml`)

    a. (Feel free to remove all lines preceded by `# @`) 
  
![Product top level keys](./images/product_top_level_keys.png)

All YAML plugins will begin with these same four properties!

In a future code cell we'll replace the YAML shown on the left of the image with the <br />
following in my_clavrx_products.yaml

```yaml
interface: products
family: list
name: my_clavrx_products
docstring: |
  CLAVR-x imagery products
```

### First Product Plugin: Cloud Top Height (CTH) from CLAVR-x data 

Edit the product specifications as shown below:

![Update product spec](./images/update_product_spec.png)

In the next code cell we'll replace the YAML shown on the upper left of the with the <br />
following in my_clavrx_products.yaml

```yaml
spec:
  products:
    - name: My-Cloud-Top-Height
      source_names: [clavrx]
      docstring: |
        CLAVR-x Cloud Top Height
      product_defaults: Cloud-Height
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"]
```

In [ ]:
"""Update the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]

updated_products["spec"]["products"] = [my_cloud_top_height]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)


In [ ]:
%%bash

cd $MY_PKG_DIR/cool_plugins/plugins/yaml/products
pwd
cat my_clavrx_products.yaml

### Command: `geoips describe`

 
Let's use the CLI to get more information about the plugin we just created. 

You'll notice that the plugin registry is rebuilt, as GeoIPS was unable to locate the 
plugin we just created (since it was not already in the registry).

In [ ]:
%%bash

geoips config create-registries
geoips describe product clavrx.My-Cloud-Top-Height

If you need a reminder what your new product is called, you can always check via:

In [ ]:
%%bash

geoips ls products -p cool_plugins

For any description of a product plugin, this is the generic format to follow via the CLI:

`geoips describe <interface_name> <source_name>.<plugin_name>`

### `family: null`

You may have noticed that your product’s family was `null` in the output of 

`geoips describe product clavrx.My-Cloud-Top-Height`. 

This isn’t the actual case. 

Products are the only plugin that can depend on another type of plugin 
(product_defaults), and that is where this information lies. 

For now, an easy way to check the family of your product is by describing its product 
default plugin. 

The family of the derived product_default is the same family as the product plugin.

In [ ]:
%%bash

geoips desc pdef Cloud-Height

### Use your new product!

To use the plugin you just created, we'll make use of the `geoips run` command.

- GeoIPS is called via a command line interface (CLI), as we've shown previously.
- The main command that you will use is `geoips run single_source`, which will run your 
  data through the specified procflow using the specified plugins 
- It's easiest to do this via a script, and scripts are stored in your plugin package's 
  `tests/` directory because they can be used later to regression test your package 
- Since we're running this in a notebook, we can easily run these commands with a code cell!

In [ ]:
%%bash

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Top-Height \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list conus

### Viewing the log output

This will write some log output.  If your script succeeded it will end with 
`INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS`.

Open the PNG file that this script produces.

If successful, the output image should look like this:

![CLAVR-x CONUS My-Cloud-Top-Height](./images/clavrx-conus-my-cloud-top-height.png)

### A word about Product Defaults 

- GeoIPS has a number of product_defaults plugins defined to help you not reinvent the 
  wheel, **but**
    - You can override any of the product defaults within your product definition 
    - You can absolutely define all of the available options within your product plugin 
- [Pre-defined CLAVR-x product defaults](https://github.com/NRLMMD-GEOIPS/geoips_clavrx/tree/main/geoips_clavrx/plugins/yaml/product_defaults) (part of the CLAVR-x plugin) 
- [Pre-defined GeoIPS product defaults](https://github.com/NRLMMD-GEOIPS/geoips/tree/main/geoips/plugins/yaml/product_defaults) 
- If you have product definition parameters that you want to reuse (i.e. if you're 
  copy/pasting product definition parameters!),consider creating a product default for 
  your plugin 

  ![Cloud-Height Product Defaults](./images/Cloud-Height-product_defaults.png)

### Different Implementations of Product Defaults within a Product 

In your product you can use the product_defaults verbatim. 

![Product using product default](./images/product_using_product_default.png)

### Different Implementations of Product Defaults within a Product 

You can also override just some parts of the product_defaults. 

In this example, we override the `algorithm` plugin contained in the Cloud-Height 
product_defaults, with our own specification. 

![Product default overridden](./images/product_default_overridden.png)

### Different Implementations of Product Defaults within a Product

We also have the option to define a product without using product_defaults.

To do this: 
  - remove the `product_defaults` property 
  - add the `family` property

![Product fully specified](./images/product_fully_specified.png)

## Hands on: Add Additional Products to Your Product Plugin 


### Add Cloud Base Height (CBH) to the CLAVR-x Product Definition

- Using your definition of `My-Cloud-Top-Height` as an example, create a product 
  definition for `My-Cloud-Base-Height`

**Helpful Hints:** 
- The relevant variable in the CLAVR-x output file (and the equivalent GeoIPS reader) 
  is called `cld_height_base`
- The Cloud-Height product_default  can be used to simplify this product 
  definition (or you can DIY or override if you'd like!) 

In [ ]:
"""Add My-Cloud-Base-Height to the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]
my_cloud_base_height = updated_products["spec"]["products"][1]

updated_products["spec"]["products"] = [my_cloud_top_height, my_cloud_base_height]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)

### First Product Plugins: CTH and CBH from CLAVR-x data (my_clavrx_products.yaml) 

After running the code cell above, the `spec` portion of `my_clavrx_products.yaml` <br />
should now look like this:

```yaml
spec:
  products: 
    - name: My-Cloud-Top-Height
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"] 
```

In [ ]:
%%bash

# verify that the contents of my_clavrx_products.yaml have the spec shown above
cd $MY_PKG_DIR/cool_plugins/plugins/yaml/products
cat my_clavrx_products.yaml

### Add Cloud Depth to the CLAVR-x Product Definition 

- Using your definitions of `My-Cloud-Top-Height` and `My-Cloud-Base-Height` as examples, 
  create a product definition for `My-Cloud-Depth`

**Helpful Hints:** 
- We will define Cloud Depth for this tutorial as the difference between CTH and CBH 

**Note:**
- This is meant to challenge you a bit! Give it a try and we'll go over the solution in 
  the next markdown block 

### Cloud Depth Product with Default Algorithm Applied (my_clavrx_products.yaml) 

After running the code block below, the `spec` portion of `my_clavrx_products.yaml` <br />
should now look like this:

```yaml
spec:
  products: 
    - name: My-Cloud-Top-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"]
    - name: My-Cloud-Depth
      source_names: [clavrx]
      docstring: |
        CLAVR-x Cloud my Cloud Depth
      product_defaults: Cloud-Height
      spec:
        variables: ["cld_height_acha", "cld_height_base", "latitude", "longitude"]
```

We now have two variables, but if we examine the Cloud-Height Product Defaults we see 
that it uses the `single_channel` algorithm.

This algorithm just manipulates a single data variable and plots it. 

Therefore, we need a new algorithm plugin!

In [ ]:
"""Add My-Cloud-Depth to the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]
my_cloud_base_height = updated_products["spec"]["products"][1]
my_cloud_depth = updated_products["spec"]["products"][2]

updated_products["spec"]["products"] = [
    my_cloud_top_height,
    my_cloud_base_height,
    my_cloud_depth,
]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)

In [ ]:
%%bash

# verify that the contents of my_clavrx_products.yaml have the spec shown above
cd $MY_PKG_DIR/cool_plugins/plugins/yaml/products
cat my_clavrx_products.yaml

### Validate your new product plugins

Before you go ahead creating your new algorithm plugin, let's validate that the product
plugins we just created are working as expected.

In [ ]:
%%bash

geoips config create-registries
geoips describe product clavrx.My-Cloud-Top-Height
geoips describe product clavrx.My-Cloud-Base-Height
geoips describe product clavrx.My-Cloud-Depth

### Command: `geoips list`

Additionally, before creating a new algorithm, let’s make sure your products are 
registered within GeoIPS. 

In [ ]:
%%bash

geoips list products -p cool_plugins

## Hands on: Create an Algorithm Plugin 

1. Copy the existing algorithm plugin to a new file to modify

In [ ]:
%%bash

cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/algorithms
cp pmw_89test.py my_cloud_depth.py

![Updating top level portions of new algorithm](./images/updating_algorithm_top_level.png)

Shown in the image above, let's update our `my_cloud_depth` algorithm with the following:

```python
"""Cloud depth product.

Difference of cloud top height and cloud base height.
"""

import logging

from xarray import DataArray

LOG = logging.getLogger(__name__)

interface = "algorithms"
family = "xarray_to_xarray"
name = "my_cloud_depth"
```

### Updating your Algorithm

Algorithms (alongside all `module-based` plugins) must include a `call()` function.

This function is what is called when the algorithm is used.

The `cal()` function's signature is determined by the algorithm's family.

To create your new algorithm, add the `"scale_factor"` parameter to the call signature.

Replace the signature of the `call()` function and its docstring with the following. You 
can remove the comments if desired.

```python
def call(
    xobj,  # Xarray Dataset holding DataArrays
    variables, # list of required input variables for the algorithm. These are ordered as specified in your product plugin.
    product_name,
    output_data_range,
    scale_factor,  # Adding a scale factor here for use in converting input meters to output kilometers
    min_outbounds="crop",
    max_outbounds="mask",
    norm=False,
    inverse=False,
):
    """My cloud depth product algorithm manipulation steps."""
```

This is where the actual data manipulation occurs. Make sure to index the variable list 
to the order of the variables you defined in your product.

Replace the contents of the `call()` function with the following. You can remove the 
comments if desired.

```python
    # Variables in the order defined in your Cloud-Depth Plugin
    cth = xobj[variables[0]]
    cbh = xobj[variables[1]]

    out = (cth - cbh) * scale_factor

    from geoips.data_manipulations.corrections import apply_data_range

    # Data manipulation: Anything you want!
    data = apply_data_range(
        out,
        min_val=output_data_range[0],
        max_val=output_data_range[1],
        min_outbounds=min_outbounds,
        max_outbounds=max_outbounds,
        norm=norm,
        inverse=inverse,
    )
    xobj[product_name] = DataArray(data)

    return xobj
```

Once complete, let's describe the plugin you just created to make sure it registered
appropriately.

In [ ]:
"""Replace the contents of my_cloud_depth.py with the correct depth algorithm."""

import os

with open("./updated_files/my_cloud_depth.py", "r") as rf:
    python_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/modules/algorithms/my_cloud_depth.py", "w") as wf:
    wf.writelines(python_lines)

In [ ]:
%%bash

geoips config create-registries
# In case you forgot the name of your plugin, you can run:
geoips ls algs -p cool_plugins
# Now let's describe that plugin
geoips describe algorithm my_cloud_depth

## Overriding the My-Cloud-Depth Algorithm

As mentioned earlier, we can override a product's defaults by adding new information to <br />
that product's `spec` object. 

This was complete when we previously added our `My-Cloud-Depth` product, however it's <br />
important to understand how this functionality works. Since that product inherits from <br />
the `Cloud-Height` product_defaults plugin, `My-Cloud-Depth` assumes it would use the <br />
`single_channel` algorithm. We want to override that with the algorithm we just created. <br />

The following YAML code demonstrates how you can override a product_default plugin in <br />
a product plugin.

```yaml
spec:
  products:
    - name: My-Cloud-Top-Height
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Top Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_acha", "latitude", "longitude"] 
    - name: My-Cloud-Base-Height 
      source_names: [clavrx] 
      docstring:  |
        CLAVR-x Cloud Base Height 
      product_defaults: Cloud-Height 
      spec:
        variables: ["cld_height_base", "latitude", "longitude"]
    # Update the code block below
    - name: My-Cloud-Depth 
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Cloud my Cloud Depth 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_acha", "cld_height_base", "latitude", "longitude"] 
        # Algorithm override portion
        algorithm: 
          plugin:
            name: my_cloud_depth # The name we assigned our algorithm when we defined it
            arguments: 
              output_data_range:  [0, 20]
              scale_factor:  0.001
```

### Hands on: Using Your Algorithm Plugin

Run the following command to make use of your new algorithm plugin, referenced in our <br />
`My-Cloud-Depth` product.

In [ ]:
%%bash

geoips run single_source \
  $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Depth \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list conus

### Using Your Algorithm Plugin 

Run your script.

This will output a bunch of log output.  

If your script succeeded it will end with `INTERACTIVE: Return Value 0`.

To view your output, look for a line that says `SINGLESOURCESUCCESS`.

Open the PNG file. It should look like this:

![My Cloud Depth Image](./images/my-cloud-depth.png)


## Part 2 -- Tutorial Contents

We're going to show you how to extend GeoIPS by adding new plugins for five new interfaces. 

We will add:  
- One new sector 
- One new feature_annotator , one new gridline_annotator 
- Three new colormappers 
- One new output_formatter

We will also show you how to: 
- Produce a NetCDF4 file using your algorithm output 
- Read your custom NetCDF4 file back in, and produce imagery from it 

### Hands on: Outputting your new product as NetCDF 

The following code block produces your `My-Cloud-Depth` product using the `netcdf_geoips`<br />
output formatter plugin. Instead of an image, this command will produce a NetCDF file <br />
using the same data as the previous image.


In [ ]:
%%bash

geoips run single_source \
   $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
   --reader_name clavrx_hdf4 \
   --product_name My-Cloud-Depth \
   --output_formatter netcdf_geoips \
   --filename_formatter geoips_netcdf_fname \
   --minimum_coverage 0 \
   --sector_list conus

There will output a bunch of log output. If your script succeeded it will end with <br />
`INTERACTIVE: Return Value 0`. To view your output, look for a line that says <br />
`SINGLESOURCESUCCESS`.

Open the NetCDF file to view its contents (e.g. with Panoply; shown below)<br />
![My Cloud Depth NetCDF Output](./images/panoply-netcdf-output.png)

**Note** - For more information on the output_formatter 
you just used, run:  

In [ ]:
%%bash

geoips describe output-formatter netcdf_geoips

### A Word About Sectors

Sectors are YAML plugins that define an area of interest for plotting in GeoIPS.

**Two types:** 
- **static**: static sector of a specific area 
- **dynamic**: sectors that are generated at run-time to follow an event (e.g. 
  tropical cyclone, atmospheric river, pyro-Cb, etc.) 

For this tutorial, we will be creating a **static**  sector. 

**Static Sector Properties:** 
- **Metadata**: Contains information related to the origin of the sector. This includes 
  continent, country, state, etc.
- **Projection**: Proj projection information 
  - **a**: The radius of the earth (in meters) 
  - **lat_0**: Center latitude coordinate 
  - **lon_0**: Center longitude coordinate 
  - **proj**: String representing projection type 
  - **units**: String representing units of the 
    projection 
- **Resolution**: The size of each pixel in meters 
- **Shape**: Image shape in pixels (width + height) 
- **Center**: The center x/y coordinates of the sector 

### Example Sectors

![Example Sectors](./images/example_sectors.png)

You can run the following commands to get more information about each of the sectors 
shown here by running:

In [ ]:
%%bash

geoips describe sector australia
geoips describe sector central_america
geoips describe sector south_pole

### Generating Static Sector Properties 

To generate the properties necessary to create a static sector, you must first define <br /> 
the area you want to display (captured by the appropriate satellite), based on center <br /> 
latitude and longitude bounds. 

The easiest way to do this is to open a [Google Maps](https://www.google.com/maps/) tab, <br /> 
then right click the center of your sector to obtain the center latitude and longitude <br /> 
values. 

Once you have those center latitude and longitude values, you’re ready to create your <br />
custom sector. 

The next slide will display the changes you will need to make to create a custom conus <br />
sector plugin. 

![Google Maps CONUS](./images/google_maps_conus.png)

**Note** - For listing of sector plugins, run: 

In [ ]:
%%bash

geoips list sectors

### Hands on: Creating Custom Sectors 

Let's start by copying a sector file to edit

In [ ]:
%%bash

# Make a sectors folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/sectors/static

# Copy a sector file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/sectors/static/australia.yaml" my_conus_sector.yaml

### Generating Static Sector Properties 

The following YAML code demonstrates how to implement your own custom sector. <br />
Since we're running this in a notebook, we'll update this file for you automatically, <br />
via the following **code block**.

You'll notice that the metadata remains untouched, however it's important to note that <br />
this section is very helpful for displaying additional information about the sector, not <br />
only for the backend of GeoIPS, but also for people using this sector plugin.

```yaml
interface: sectors
family: area_definition_static 
name: my_conus_sector 
docstring: "My CONUS Sector" 
metadata: 
  region: 
    continent: NorthAmerica 
    country: UnitedStates 
    area: x
    subarea: x
    state: x
    city: x
spec:
  area_id: my_conus_sector 
  description: CONUS
  projection: 
    a: 6371228.0 
    lat_0: 37.0
    lon_0: -96.0
    proj: eqc
    units: m
  resolution: 
    - 3000
    - 3000
  shape: 
    height: 1000
    width: 2200
  center: [0, 0]
```


In [ ]:
"""Update my_conus_sector with the correct contents."""

import os

with open("./updated_files/my_conus_sector.yaml", "r") as yaml_file:
    updated_sector = yaml.safe_load(yaml_file)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/sectors/static/my_conus_sector.yaml", "w") as f:
    yaml.safe_dump(updated_sector, f, default_flow_style=False, sort_keys=False)

### Testing Your Custom Static Sector 

The commands you ran in the previous slide create a custom conus sector. <br />
`my_conus_sector.yaml` will be an  example plugin, showing you that you can create sectors <br />
just like `conus.yaml`, to your own specifications. 

To quickly check whether or not you like the shape and resolution of your custom sector, <br />
you can use the command line function create_sector_image . This will plot and save <br />
images containing the borders and coastlines of the inputted sectors. For example, to test <br />
your custom sector, run the following: 

In [ ]:
%%bash

geoips config create-registries
geoips test sector my_conus_sector

Once completed, open the `my_conus_sector.png`  image to see what your sector will look like. <br />
It should look like this:

![My CONUS Sector](./images/my_conus_sector.png)

**Note** -  You can describe the sector plugin you just 
created by running:  

In [ ]:
%%bash

geoips describe sector my_conus_sector

### Using Your Custom Static Sector

To use `my_conus_sector.yaml` in your test script, simply replace `--sector_list conus` <br />
with `--sector_list my_conus_sector`.

Let's run a command that now uses your new sector plugin.


Let's run that script you just created and modified.

In [ ]:
%%bash

geoips run single_source \
    $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Top-Height \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list my_conus_sector

The result of executing that script should look like this:

![CTH my conus sector](./images/my_conus_sector_cloud_top_height.png)

## Hands on: Creating Custom Gridline/Feature Annotators 

### Defining gridline_annotators and feature_annotators 

**gridline_annotators:**
- Describe the format of the grid lines shown in your imagery. Can control: 
- **Labels**, and which to display 
- **Lines**, such as their **color**, **linestyle**, and **linewidth** 
- **Spacing**, such as the distance between **latitude** and **longitude** labels, and 
  the grid lines that represent them. 

**feature_annotators:**
- Describe the format of the features shown in your imagery. Can control: 
- **Coastlines**, **Borders**, **States**, and **Rivers**. All of these features have 
  the same parameters: 
  - Whether they are **enabled** 
  - The **color** of the feature 
  - The **linewidth** of the feature displayed 

### Example Gridline and Feature Annotators 

![Example gridline and feature annotators](./images/example_gridline_and_feature_annotators.png)

You can describe the aforementioned plugins via:

In [ ]:
%%bash

geoips describe feature-annotator default
geoips describe gridline-annotator default

### Creating a Custom gridline_annotator 

Let's begin by setting up a directory for your gridline annotators and copy over an 
existing plugin from GeoIPS.

In [ ]:
%%bash

# Make a gridline_annotators folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/gridline_annotators
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/gridline_annotators

# Copy a gridline_annotator file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/gridline_annotators/default.yaml" tutorial.yaml

### Breaking down a gridline_annotator

The following image explains each component of a gridline_annotator plugin.

![Gridline Annotator Explained](./images/gridline_annotator_explained.png)

Now that you understand what a gridline_annotator plugin does, we'll modify the plugin we <br />
copied over. 

Since we're running in a notebook, we'll automatically do this for you. However, please <br />
take in the contents of `tutorial.yaml` with the YAML shown below.

```yaml
interface: gridline_annotators 
family: cartopy
name: tutorial 
docstring: |
  The tutorial gridline_annotators configuration.  
  All gridline labels enabled, latitude and 
  Longitude lines colored mediumseagreen, 2.5 degree 
  spacing, 1px linewidth, and [5, 3] linestyle. 
spec:
  labels:  
    top: true
    bottom: true
    left: true
    right: true
  lines:  
    color: mediumseagreen 
    linestyle: [5, 3]
    linewidth: 1
  spacing:  
    latitude: 2.5
    longitude: 2.5
```

In [ ]:
"""Update gridline_annotator - tutorial.yaml with the correct contents."""

import os

with open("./updated_files/gridline_annotators/tutorial.yaml", "r") as yaml_file:
    updated_gridline_annotator = yaml.safe_load(yaml_file)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/gridline_annotators/tutorial.yaml", "w") as f:
    yaml.safe_dump(updated_gridline_annotator, f, default_flow_style=False, sort_keys=False)

### Creating custom feature_annotator 

Now that we've created a custom gridline_annotator plugin, let's do the same for a <br />
feature_annotator plugin.

In [ ]:
%%bash

# Make a feature_annotators folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/feature_annotators
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/yaml/feature_annotators

# Copy a feature_annotator file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/yaml/feature_annotators/default.yaml" tutorial.yaml

### Breaking down a feature_annotator

The following image explains how a feature is set up in a feature_annotator plugin.

You'll notice that a feature can either be disabled, and if not, you must define its <br /> 
color and linewidth.

![Feature Annotator Explained](./images/feature_annotator_explained.png)

Now that you understand what a feature_annotator plugin does, we'll modify the plugin we <br />
copied over. 

Since we're running in a notebook, we'll automatically do this for you. However, please <br />
take in the contents of `tutorial.yaml` with the YAML shown below.


```yaml
interface: feature_annotators
family: cartopy
name: tutorial 
docstring: |
  The tutorial feature annotators configuration.  
  All line types enabled. All colored  
  [midnightblue, red, yellow, cyan]. 2px coastlines, 
  2px borders, 2px states, 1px rivers .
spec:
  coastline:  
    enabled: true
    edgecolor : midnightblue 
    linewidth : 2
  borders:  
    enabled: true
    edgecolor: red
    linewidth: 2
  states:  
    enabled: true
    edgecolor: darkslategray 
    linewidth: 2
  rivers:  
    enabled: true
    edgecolor: cyan
    linewidth: 1
```

In [ ]:
"""Update feature_annotators - tutorial.yaml with the correct contents."""

import os

with open("./updated_files/feature_annotators/tutorial.yaml", "r") as yaml_file:
    updated_feature_annotator = yaml.safe_load(yaml_file)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/feature_annotators/tutorial.yaml", "w") as f:
    yaml.safe_dump(updated_feature_annotator, f, default_flow_style=False, sort_keys=False)

### Using Your Custom Annotators

To add the effects of these plugins to your final output, simply add the following lines <br /> 
to your command that uses the `"imagery_annotated”` output formatter.

```bash
--feature_annotator tutorial \
--gridline_annotator tutorial \
```

Once you've added those, you can run the script via:

In [ ]:
%%bash

# build the new plugin registries
geoips config create-registries

geoips run single_source $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Depth \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --feature_annotator tutorial \
  --gridline_annotator tutorial \
  --sector_list conus

If you copied the YAML for the gridline and feature -annotators verbatim, the result of <br />
that script should look like this:

![Feature / Gridline annotator image](./images/feature_gridline_annotator.png)

## Colorbars and Colormaps Background and Introduction 


### Using a Matplotlib Colormap 

Matplotlib has many predefined colormaps.

The `matplotlib_linear_norm` colormapper plugin is capable of using these predefined <br />
colormaps - simply provide the name of the colormap in the `“cmap_name”` argument when <br />
you define your product plugin.

For now `matplotlib_linear_norm` is the only plugin that can directly use named <br />
colormaps (which explicitly normalizes the colors using matplotlib’s `“Normalize”` <br />
function), though others may be added in the future with additional methods of <br />
manipulating the color scales.

The image below displays a sample product plugin using matplotlib built-in colormap.

![Product using matplotlib_linear_norm](./images/product_using_mpl_linear_norm.png)

### Using an ASCII Colormap 

The `matplotlib_linear_norm` plugin can also leverage `ASCII` colormap files installed <br />
within GeoIPS, installed within a plugin package, or stored in an arbitrary location on disk. 

To use an ascii colormap, specify:
  - `cmap_source: ascii` 
  - `cmap_name: <cmap_name>`

  
Installed ASCII colormaps are stored within a plugin package under `plugins/txt/ascii_palettes/`.
  - The filename must be of the form `<cmap_name>.txt` (this is how geoips finds the given <br />
    `cmap_name`)
  - The colors are defined as RGB triplets ranging from 0 to 256 and should be formatted <br /> 
    as three columns of 256 integers. 
  - Commented lines are allowed (prefaced by “#”), to provide additional context relating <br />
    to the physical meaning of the colormap (ie, min and max expected values/units, <br />
    transition points, etc) 
  - For an example, see the [tpw_purple](https://github.com/NRLMMD-GEOIPS/geoips/blob/710b58ae82f479168e7da49dfccd8650730478d8/geoips/plugins/txt/ascii_palettes/tpw_purple.txt) colormap in the geoips package. 

If you would like to specify an arbitrary full path on disk rather than installing your <br />
ascii palette within your  plugin package, additionally specify cmap_path . This is most <br />
useful for research, development, and testing purposes. 
  - `cmap_path: <full_path>/<any_name>.txt`

`matplotlib_linear_norm` provides a number of options that can be used to customize how <br />
the colormap is used and how the colorbar is drawn. 

The following image displays a sample product using ascii palette -based colormap.

![Product using ascii colormap](./images/product_using_ascii_colormap.png)

### Using a custom GeoIPS Python-based colormapper

Color information can also be specified via a python-based GeoIPS `“colormapper”` plugin, <br />
allowing customization using specific matplotlib commands and utilities. <br />
This is the most flexible method.

The following images walk you through each part of a colormapper plugin.

![Colormapper top level](./images/colormapper_top_level.png)

![Colormapper inside call function](./images/colormapper_inside_call_function.png)

The `“mpl_colors_info”` return dictionary is what GeoIPS uses within the matplotlib-based <br />
utilities and output formatters in order to ensure consistent application of colors, in <br />
both the imagery and the colorbars. 

Coming up, we'll walk you through creating both GeoIPS Python-based colormapper, as well <br />
as ASCII-palette based matplotlib_linear_norm products.

![Colormapper mpl_colors_info](./images/mpl_colors_info.png)

## Hands on: Creating your own custom Python-based colormapper 

Let's begin by creating a directory to hold our colormappers and copying over an existing <br />
colormapper from GeoIPS.

In [ ]:
%%bash

# Make a colormappers folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/colormappers
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/colormappers

# Create an __init__.py file
touch __init__.py

# Copy a colormapper file from GeoIPS
geoips_path="$(pip show geoips_clavrx | awk '/^Location: / {print $2 "/geoips_clavrx"}')"
cp "$geoips_path/plugins/modules/colormappers/cmap_cldHeight.py" colorful_cloud_height.py

Now, lets update the file we copied over with new content. We'll replace the upper <br />
portions of `colorful_cloud_height.py` with the following python code.

Note: Since we're using a notebook, this will be done for you in the following code block.

```python
"""Module containing colormap for colorful cloud height product.""" 

import logging

LOG = logging.getLogger(__name__) 

interface = "colormappers" 
family = "matplotlib" 
name = "colorful_cloud_height" 


def call(data_range=[ 0, 20]):
    """Colorful cloud height colormap."""
```

Once that's complete, let's update the contents of the `call()` function to change the <br />
colormap we'll produce. Feel free to choose different colors if you'ld like.

In this case, we only modify the colors, but you just as easily could modify the transition <br />
values as well. These are all just direct matplotlib/python commands creating the desired <br />
color information.

```python
    transition_vals = [
        (min_val, 1),
        (1, 2),
        (2, 3),
        (3, 4),
        (4, 6),
        (6, 8),
        (8, 10),
        (10, 15),
        (15, max_val),
    ]
    transition_colors = [
        ("pink", "red"),
        ("paleturquoise", "teal"),
        ("plum", "rebeccapurple"),
        ("yellow", "chartreuse"),
        ("limegreen", "darkgreen"),
        ("wheat", "darkorange"),
        ("darkgray", "black"),
        ("lightgray", "silver"),
        ("lightskyblue", "deepskyblue"),
    ]
```

In [ ]:
"""Replace the contents of colorful_cloud_height.py with the correct colormapper information."""

import os

with open("./updated_files/colorful_cloud_height.py", "r") as rf:
    python_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/modules/colormappers/colorful_cloud_height.py", "w") as wf:
    wf.writelines(python_lines)

## Hands on: Using your new python-based colormapper in a new product

### Add custom colormapper to new product. 

Let's begin by creating a new `Cloud-Base-Python-Colors` product, using `Cloud-Height` <br />
as the product_defaults.

We'll do this automatically in the next code cell.

Normally, you'd just add the following product entry to your `my_clavrx_products.yaml` <br />
file. Note that it makes use of the colormapper we just put together!

```yaml
    - name: Cloud-Base-Python-Colors
      source_names:  [clavrx] 
      docstring:  |
        CLAVR-x Colorful Cloud Base Height, 
        Using a python-based custom colormapper. 
      product_defaults:  Cloud-Height 
      spec:
        variables:  ["cld_height_base", "latitude", "longitude"] 
        colormapper: 
          plugin: 
            name: colorful_cloud_height 
            arguments: {}
```

In [ ]:
"""Add My-Cloud-Depth to the contents of my_clavrx_products.yaml."""

import os

import yaml

with open("./updated_files/my_clavrx_products.yaml", "r") as yaml_file:
    updated_products = yaml.safe_load(yaml_file)

my_cloud_top_height = updated_products["spec"]["products"][0]
my_cloud_base_height = updated_products["spec"]["products"][1]
my_cloud_depth = updated_products["spec"]["products"][2]
cloud_base_python_colors = updated_products["spec"]["products"][3]

updated_products["spec"]["products"] = [
    my_cloud_top_height,
    my_cloud_base_height,
    my_cloud_depth,
    cloud_base_python_colors,
]

print(updated_products)

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/yaml/products/my_clavrx_products.yaml", "w") as f:
    yaml.safe_dump(updated_products, f, default_flow_style=False, sort_keys=False)

Let's modify the following line of our previous product command with the following.

```bash
--product_name My-Cloud-Top-Height \
```

with

```bash
--product_name Cloud-Base-Python-Colors \
```

Now, you can produce your new product which uses the colormapper we created!

In [ ]:
%%bash

# rebuild your registry files
geoips config create-registries
geoips run single_source $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name Cloud-Base-Python-Colors \
  --output_formatter imagery_annotated \
  --filename_formatter geoips_fname \
  --minimum_coverage 0 \
  --sector_list conus

You should get the following image from that script:

![Colorful Cloud Height](./images/colorful_cloud_height.png)

## Hands on: Creating a new Output Formatter

### What are output formatters? 

- Output formatters take native GeoIPS xarray data and convert it into something interpretable! 
    - NetCDF, an image, HDF5, <you name it!>
- Required arguments depend on the intent of your output!

You can see what arguments are required for each family of output formatters [here](https://github.com/NRLMMD-GEOIPS/geoips/blob/main/geoips/interfaces/module_based/output_formatters.py).

You'll notice that output formatters have few to many arguments, depending on the family <br />
they inherit from.

![Easy output formatter](./images/easy_output_formatter.png)

![Hard output formatter](./images/hard_output_formatter.png)

### Making your own output formatter

Now, let's create our own output formatter. We'll begin by creating a directory for these <br />
plugins and copying over an existing output formatter from GeoIPS.

In [ ]:
%%bash

# Make a output_formatters folder and change directories into it
mkdir -pv $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters
cd $MY_PKG_DIR/$MY_PKG_NAME/plugins/modules/output_formatters

# Create an __init__.py file
touch __init__.py

# Copy a output_formatter file from GeoIPS
geoips_path="$(pip show geoips | awk '/^Location: / {print $2 "/geoips"}')"
cp "$geoips_path/plugins/modules/output_formatters/netcdf_geoips.py" my_netcdf_output.py

Now that we have our own output formatter file, let's edit it.

We'll do this automatically for you, but please take in what this product does.

`Step 1: Update this section` 

```python
interface = "output_formatters"
family = "xarray_data"
name = "my_netcdf_output"
```

`Step 2: Add some new attributes!`

```python
# This is inside the call() function
copy_standard_metadata (xarray_obj, prod_xarray) 
for product_name in product_names: 
    prod_xarray[product_name] = xarray_obj[product_name] 
# Let's add some additional attributes that are necessary for us 
# Add the section below! 
prod_xarray = prod_xarray.assign_attrs (
    Starring= "Richard Karn" ,
    Featuring= "Jonathan Taylor Thomas" ,
    ProducedBy= "Carmen Finestra" ,
)
```

Reminder: you can always review your plugins via `geoips describe`.

In [ ]:
"""Replace the contents of my_netcdf_output.py with the correct output_formatter information."""

import os

with open("./updated_files/my_netcdf_output.py", "r") as rf:
    python_lines = rf.readlines()

with open(f"{os.environ['MY_PKG_DIR']}/cool_plugins/plugins/modules/output_formatters/my_netcdf_output.py", "w") as wf:
    wf.writelines(python_lines)

In [ ]:
%%bash

# rebuild your registry files
geoips config create-registries
geoips describe output-formatter my_netcdf_output

### Using your new output_formatter

Let's make use of our new output_formatter running the following command.

All that's modified is the following:

```bash
--output_formatter netcdf_geoips \
```

with

```bash
--output_formatter my_netcdf_output \
```

Run your new script!

In [ ]:
%%bash

geoips run single_source $GEOIPS_TESTDATA_DIR/test_data_clavrx/data/goes16_2023101_1600/clavrx_OR_ABI-L1b-RadF-M6C01_G16_s20231011600207.level2.hdf \
  --reader_name clavrx_hdf4 \
  --product_name My-Cloud-Depth \
  --output_formatter my_netcdf_output \
  --filename_formatter geoips_netcdf_fname \
  --minimum_coverage 0 \
  --sector_list conus

In [ ]:
import os

import xarray

fpath = (
    f"{os.environ['GEOIPS_OUTDIRS']}/preprocessed/algorithms/My-Cloud-Depth_latitude_lo"
    "ngitude/clavrx/goes-16/conus/20230411/20230411.160020.goes-16.My-Cloud-Depth_latit"
    "ude_longitude.conus.nc"
)

ds = xarray.open_dataset(fpath)

ds.attrs

The produced file should look like this:

![NetCDF Output](./images/netcdf_output.png)

## A word about Readers 


### Readers Overview - Content

- GeoIPS readers return a dictionary of xarrays minimally containing the following variables:


| Common Name(s) | Required name in xarray | Data format |
| :------------: | :---------------------: | :---------: |
| Latitude, Longitude | latitude, longitude | float or int |
| Variables of Interest | **<customizable but should match <br /> names used by <br /> products!>** | float or int |
| Time of Observation | time | datetime object |
| Metadata attribute: Data Source | **source_name** | string |
| Metadata attribute: Time of First <br /> Observation | start_datetime | datetime object |
| Metadata attribute: Time of Final <br /> Observation | end_datetime | datetime object |

### Why are those bolded ones so important? 

- Those two determine how GeoIPS interfaces with your reader and product! 

![Reader Bolded Items Explained](./images/reader_bolded_items.png)

### Reader overview - Structure 

- A GeoIPS reader typically consists of the following: 
    - `The call function`<br />
      The call function is the main driver of a GeoIPS reader. It accepts the kwargs that <br />
      contain the list of files to be read, and a handful of instructions that adjust how <br />
      the reader functions. 
    - `One or several read functions`<br />
      Read functions populate xarrays with data from the files themselves. Read functions <br />
      are technically optional... but they are a best practice! 
    - `Optional utility functions`<br />
      Utility functions perform some kind of operation on inputs, typically to convert <br />
      them to a format understandable by GeoIPS 

![Reader Functions](./images/reader_functions.png)

### *call* function key points 

![Reader call function key points](./images/reader_call_function_key_points.png)

### A typical *read* function 

- Largely a *dealer’s choice* (and *you’re* the dealer!) 
    - The read function needs to open the file and read the contents (remember the minimum <br />
      content slide) into a dictionary of xarrays to be passed along to GeoIPS 
- Common challenges to be aware of: 
    - 1D variables: It’s okay if your variables are 1D... so long as *all of them* are 1D.<br /> 
      You may need to do some array manipulation to get everything even! This is a common <br />
      issue particularly with times! 
    - Seaking of times, time formatting is a common issue: TAI93… UTC… binary string... <br />
      seconds since epoch… there are a lot of ways time is reported in data! Consult the <br /> 
      user’s guide for your data to figure out what how to convert time variables to the <br />
      required datetime object format. 
    - Reading in necessary channels: GeoIPS cannot intelligently read required channels <br />
      unless you code your reader to do so. Remember your call script is invoked with the <br />
      *chans* parameter– use that information to save you and your customer’s time! 


### An Example Read Function from GMI (1) 

![GMI Read Function](./images/gmi_read_function.png)

Continued...

![GMI Read Function 2](./images/gmi_read_function2.png)

One last cool thing you can do!

![Vertical data stacking](./images/vertical_stack_data.png)

## Wrap Up

### Additional Information 

- Bonus exercises can be found in the [slide package](https://docs.google.com/presentation/d/1tkdRUFjZh_AdA98G1jIjaEjXn_HjYQD3PcweVmTUmTo/edit?slide=id.g224c8803d80_0_82#slide=id.g224c8803d80_0_82) 
    - Read that NetCDF4 file to produce imagery 
    - Change which colormap you are using to create your imagery 
- GeoIPS Slack Workspace<br />
  https://geoips.slack.com/  


### Handy Links 

- Solution repository 
    - Github: https://github.com/NRLMMD-GEOIPS/plugin_tutorial_solution  
- GeoIPS repository: https://github.com/NRLMMD-GEOIPS/geoips 